# Inference using packages

Differen gpt2 model options
- gpt2: This is the "small" version of GPT-2. It has 124 million parameters.
- gpt2-medium: This is the "medium" version of GPT-2. It has 355 million parameters.
- gpt2-large: This is the "large" version of GPT-2. It has 774 million parameters.
- gpt2-xl: This is the "extra large" version of GPT-2. It has 1.5 billion parameters.


In [100]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
gpt2 = GPT2LMHeadModel.from_pretrained('gpt2') # loading gpt2 from transformers library
gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2') # loading gpt2 tokenizer from transformers library
# print(gpt2)

In [101]:
input_text = "A long time ago in a galaxy far far away ..."
input_text = "music is"
input_ids = gpt2_tokenizer.encode(input_text, return_tensors='pt') # tokenize input
output = gpt2.generate(input_ids, max_length=100) # run inference
generated_text = gpt2_tokenizer.decode(output[0], skip_special_tokens=True) # decode output tokens
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


music is a great way to get a sense of what's going on in your life.

I think it's important to understand that you're not just talking about the music. You're talking about the people who are listening to it.

I think that's a good thing. I think that's a good thing. I think that's a good thing. I think that's a good thing. I think that's a good thing. I think that's a good thing. I


# Inference using Numpy

In [102]:
state_dict = gpt2.state_dict()
parameters = {}
for name, val in state_dict.items():
    parameters[name] = val.numpy()


for name, param in state_dict.items():
    ans = param.numpy()
    if 'h.' not in name: # each h.# refers to a transformer blocks
        # print(f'{name}: {ans.shape}')
        pass

for i in range(12):
    counter = 0
    for name, param in state_dict.items():
        ans = param.numpy()
        if 'h.'+ str(i)+ '.' in name: # each h.# refers to a transformer block
            # print(f'{name}: {ans.shape}')
            counter +=1
    # print(f'h.{i}: {counter}')

[0 1 2 3 4 5 6 7 8 9]


In [121]:
import numpy as np
import heapq
import random

parameters = parameters

def torch_to_numpy(tensor): # not nessessarry?
    if tensor.is_cuda:
        tensor = tensor.cpu()
    numpy_array = tensor.numpy()
    return numpy_array.copy()

# TODO which softmax?
def softmax(vec): # the, and
    max_val = np.max(vec)
    exp = np.exp(vec - max_val)
    sum_exp = np.sum(exp)
    norm_vec = exp/sum_exp
    assert 0.975 < np.sum(norm_vec) < 1.025
    return norm_vec

def log_softmax(vec, epsilon=1e-12): # puncuation
    max_val = np.max(vec)
    exp = np.exp(vec - max_val)
    log_sum_exp = max_val + np.log(np.sum(exp))
    return vec - log_sum_exp

def repo_softmax(vec): # error
    x = vec - np.argmin(vec)
    ex = np.exp(x)
    return ex/ np.sum(ex)

# activation functions
def gelu(x):
    # from https://github.com/openai/gpt-2.git
    # x(np_array) Gausien error liner unit
    return 0.5*x*(1+np.tanh(np.sqrt(2/np.pi)*(x+0.044715*np.power(x, 3))))

def ReLU(x):
    # x(np_array) clip negitive activation
    return np.maximum(0, x)

# Transformer functions
def embed(tok):
    '''
    creat embeding matrix (token, token embeding vector 768)
    tok(np_array): 1d array of toek encodings
    paramaters(dict): dictionary maping names to tensors
    '''
    # word token embeddings
    tok_emb = parameters['transformer.wte.weight'][tok,:]

    # word position embeddings
    sequence_length = tok.shape[0]
    position_ids = np.arange(sequence_length) #indicies
    position_emb = parameters['transformer.wpe.weight'][position_ids,:]
    assert tok_emb.shape == position_emb.shape
    return tok_emb + position_emb

def li_norm(x, gamma, beta, epsilon=1e-5):
    '''
    layer batch normalization
    x(np_array): array to normalize
    gamma(np_array): scailing paramater vector
    beta(np_array): offset paramater vector
    epsilon(float): div_0_error prevention
    '''
    u = np.mean(x, axis=-1, keepdims=True)
    # s = np.var(x, axis=-1, keepdims=True)
    s = np.mean(np.square(x-u))
    x = (x - u) / np.sqrt(s + epsilon)
    return x*gamma + beta

def get_head_weights(head_tot, weights, bias):
    '''
    head_tot(int)
    weights(np.matrix) (tok)
    bias(np.vec)
    return ittorator (tup): head_w, head_b
    '''
    head_width = int(weights.shape[1] / head_tot)
    start = 0
    end = head_width
    for _ in range(head_tot):
        yield (weights[:, start:end], bias[start:end])
        start += head_width
        end += head_width

def split_head(head_param):
    '''
    head_param (tup): head_w, head_b
    '''
    weights, bias = head_param
    width = int(head_param[1].shape[0] / 3)

    start = 0
    end = width
    for _ in range(3):
        yield (weights[:, start:end], bias[start:end])
        start += width
        end += width

prev_QKV = {}

def self_attn(emb, block_num):
    '''
    attention block. 12 heads per block
    emb(np_matrix): (tokens, Embedding Size 768)
    paramaters(dict): dictionary maping names to tensors
    block_num: current head
    '''
    attn_weights = parameters['transformer.h.'+ str(block_num) + '.attn.c_attn.weight']
    attn_bias = parameters['transformer.h.'+ str(block_num) + '.attn.c_attn.bias']

    context_matrix = np.empty((emb.shape[0],0))
    for head_num, head_param in enumerate(get_head_weights(12, attn_weights, attn_bias)):
        QKV_gen = split_head(head_param)
        query_w, query_b = next(QKV_gen)
        key_w, key_b = next(QKV_gen)
        value_w, value_b = next(QKV_gen)

        if f'q_b{block_num}_h{head_num}' in prev_QKV:
            new_emb = emb[-1]
            np.vstack((prev_QKV[f'q_b{block_num}_h{head_num}'], new_emb @ query_w + query_b))
            np.vstack((prev_QKV[f'k_b{block_num}_h{head_num}'], new_emb @ key_w + key_b))
            np.vstack((prev_QKV[f'v_b{block_num}_h{head_num}'], new_emb @ value_w + value_b))
            Q = prev_QKV[f'q_b{block_num}_h{head_num}']
            K = prev_QKV[f'k_b{block_num}_h{head_num}']
            V = prev_QKV[f'v_b{block_num}_h{head_num}']
        else:
            Q = np.apply_along_axis(lambda x: x + query_b, axis = 1, arr=emb @ query_w)
            K = np.apply_along_axis(lambda x: x + key_b, axis = 1, arr=emb @ key_w)
            V = np.apply_along_axis(lambda x: x + value_b, axis = 1, arr=emb @ value_w)

            prev_QKV[f'q_b{block_num}_h{head_num}'] = Q
            prev_QKV[f'k_b{block_num}_h{head_num}'] = K
            prev_QKV[f'v_b{block_num}_h{head_num}'] = V

        # Q = np.apply_along_axis(lambda x: x + query_b, axis = 1, arr=emb @ query_w)
        # K = np.apply_along_axis(lambda x: x + key_b, axis = 1, arr=emb @ key_w)
        # V = np.apply_along_axis(lambda x: x + value_b, axis = 1, arr=emb @ value_w)

        attn_score = Q @ K.T

        attn_score = attn_score / np.sqrt(emb.shape[1])

        # future_mask
        future_mask = np.full(attn_score.shape, 0.0)
        future_mask[np.triu_indices_from(future_mask, k=1)] = float('-inf')

        attn_score_mask = attn_score + future_mask

        attn_score_norm = np.apply_along_axis(softmax, axis=1, arr=attn_score_mask)

        context_matrix = np.hstack([context_matrix, attn_score_norm @ V])

    return context_matrix

def mlp(emb, block_num):
    '''
    2 layer multi layer perceptron with gelu activation
    emb(np_matrix): (tokens, Embedding Size 768)
    paramaters(dict): dictionary maping names to tensors
    block_num: current head
    '''
    weights = parameters['transformer.h.'+ str(block_num) + '.mlp.c_fc.weight']
    bias = parameters['transformer.h.'+ str(block_num) + '.mlp.c_fc.bias']
    embl1 = (emb @ weights) + bias

    embl1 = gelu(embl1)

    weights = parameters['transformer.h.'+ str(block_num) + '.mlp.c_proj.weight']
    bias = parameters['transformer.h.'+ str(block_num) + '.mlp.c_proj.bias']
    return (embl1 @ weights) + bias

def top_k(k, vec):
    largest = heapq.nlargest(k, range(len(vec)), vec.take)
    # print(gpt2_tokenizer.decode(largest, skip_special_tokens=True)) # see words its picking from.
    probs = [vec[i] for i in largest]
    return random.choices(largest, weights=probs, k=1)[0]


In [122]:
parameters = parameters

def decode_block(emb, block_num):
    '''
    runs decode block with ln_1 -> attn -> ln_2 -> mlp
    emb (np_array): (tokens, Embedding Size 768)
    paramaters(dict): dictionary maping names to tensors
    block_num: current head
    '''
    # ln_1 normalization
    weights = parameters['transformer.h.'+ str(block_num) + '.ln_1.weight']
    bias = parameters['transformer.h.'+ str(block_num) + '.ln_1.bias']
    emb_norm1 = li_norm(emb, weights, bias)


    context_matrix = self_attn(emb_norm1, block_num)
    context_matrix += emb # Residual Connection

    # ln_2 normalization
    weights = parameters['transformer.h.'+ str(block_num) + '.ln_2.weight']
    bias = parameters['transformer.h.'+ str(block_num) + '.ln_2.bias']
    emb_norm2 = li_norm(context_matrix, weights, bias, epsilon=1e-5)

    emb_mlp = mlp(emb_norm2, block_num)
    emb_mlp += emb_norm2    # Residual Connection
    return emb_mlp


In [123]:
def next_token(tok):
    '''
    Generates the next token in sequence
    tok (np_array): 1D token encodigns
    parameters(dict): dictionary maping names to tensors
    '''
    emb = embed(tok) #(tokens, Embedding Size 768)

    block_result = emb
    for block in range(12): # for every decode block
        block_result = decode_block(block_result, block) # (tokens, Embedding Size 768)

    # ln_f
    weights = parameters['transformer.ln_f.weight']
    bias = parameters['transformer.ln_f.bias']
    head_norm = li_norm(block_result, weights, bias)

    # lm_head
    weights = parameters['lm_head.weight'] # (50257, 768)
    logit_matrix = head_norm @ weights.T

    # apply softmax to last words logit
    last_logit_distrabution = softmax(logit_matrix[-1])
    return top_k(5, last_logit_distrabution)


In [124]:
def main(prompt, max_token_len = 1024):
    '''
    creates generation feedback loop
    prompt(srt)
    start_dict(dict): name: paramaters
    '''
    gpt2_tokenizer.pad_token = gpt2_tokenizer.eos_token
    prompt_tok_len = len(gpt2_tokenizer.encode(prompt, return_tensors='np').squeeze())
    tok = gpt2_tokenizer.encode(prompt, return_tensors='np', padding='max_length', truncation=True, max_length=max_token_len)
    tok = tok.squeeze()

    for i in range(10):
        if prompt_tok_len < max_token_len:
            new_tok = next_token(tok)
            tok[prompt_tok_len] = new_tok
            prompt_tok_len += 1


    print(tok[:10])
    token_decoded = gpt2_tokenizer.decode(tok, skip_special_tokens=True)
    return token_decoded

print(main('waht do you think, thinking machiene?'))

[  86  993   83  466  345  892   11 3612 3235   72]
waht do you think, thinking machiene? the the the the the the you the the the
